# Advanced Customer LTV Analysis with In-Database Feature Engineering

**Objective:** Predict Customer Lifetime Value (LTV) by creating advanced features directly in Snowflake. This notebook uses a complex SQL query with CTEs and window functions to aggregate customer behavior before training a model.

In [ ]:
import pandas as pd
import snowflake.connector
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

print("Libraries loaded for advanced analysis.")

In [ ]:
# Establish connection to Snowflake
conn = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    warehouse='ANALYST_WH',
    database='RETAIL_PROD',
    schema='ANALYTICS'
)
print("Snowflake connection established.")

In [ ]:
# This single, complex query performs all feature engineering in Snowflake.
feature_engineering_sql = """
WITH customer_transactions AS (
    -- First, get transaction details and calculate days since last purchase
    SELECT
        CUSTOMER_ID,
        TRANSACTION_ID,
        TRANSACTION_DATE,
        AMOUNT,
        DATEDIFF('day', TRANSACTION_DATE, CURRENT_DATE) as DAYS_SINCE_TRANSACTION
    FROM TRANSACTIONS
),

customer_aggregates AS (
    -- Next, aggregate key metrics for each customer
    SELECT
        CUSTOMER_ID,
        COUNT(TRANSACTION_ID) as TOTAL_TRANSACTIONS,
        SUM(AMOUNT) as TOTAL_SPEND,
        AVG(AMOUNT) as AVG_TRANSACTION_VALUE,
        MIN(DAYS_SINCE_TRANSACTION) as RECENCY -- Days since their most recent purchase
    FROM customer_transactions
    GROUP BY CUSTOMER_ID
),

customer_cohorts AS (
    -- Finally, join with customer info and add window function features
    SELECT
        c.CUSTOMER_ID,
        c.TENURE_MONTHS,
        c.PREFERRED_CHANNEL,
        agg.TOTAL_TRANSACTIONS,
        agg.TOTAL_SPEND as LTV_TARGET, -- This is our prediction target
        agg.AVG_TRANSACTION_VALUE,
        agg.RECENCY,
        -- Calculate percentile rank for spend within their channel cohort
        PERCENT_RANK() OVER (PARTITION BY c.PREFERRED_CHANNEL ORDER BY agg.TOTAL_SPEND) as SPEND_PERCENTILE_IN_CHANNEL
    FROM CUSTOMERS c
    JOIN customer_aggregates agg ON c.CUSTOMER_ID = agg.CUSTOMER_ID
)

SELECT * FROM customer_cohorts;
"""

print("Executing advanced feature engineering query in Snowflake...")
ltv_features_df = pd.read_sql(feature_engineering_sql, conn)
conn.close()
print(f"Loaded {len(ltv_features_df)} customer feature sets.")

In [ ]:
# Prepare data for modeling: One-hot encode categorical features
df_model_data = pd.get_dummies(ltv_features_df, columns=['PREFERRED_CHANNEL'], drop_first=True)
df_model_data.set_index('CUSTOMER_ID', inplace=True)

# Define features (X) and target (y)
X = df_model_data.drop('LTV_TARGET', axis=1)
y = df_model_data['LTV_TARGET']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data prepared for modeling.")

In [ ]:
# Train a Gradient Boosting Regressor model
print("Training LTV prediction model...")
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbr.fit(X_train, y_train)

print("Model training complete.")

# Evaluate the model
predictions = gbr.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"Model evaluation complete. Test RMSE: ${rmse:,.2f}")